# Xarray-spatial
### User Guide: Zonal crosstab
-----

Xarray-spatial's zonal crosstab function provides an easy way to calculate cross-tabulated (categorical stats) areas between two datasets that can help in identifying patterns and trends in the data. In this notebook, we'll analyze temperature by elevation. We use temperature data from [MODIS](https://planetarycomputer.microsoft.com/dataset/modis-21A2-061) dataset. The elevation data is from [NASADEM](https://planetarycomputer.microsoft.com/dataset/nasadem) dataset. Both the 2 are coutinuous data. To categorize them, we'll group elevation into different classes so that elevation of similar height will be in the same class. Similarly, temperature will also be classified into different classes. We'll use zonal crosstab in 2 different scenarios:


[2D Zonal Crosstab](#2D-Zonal-Crosstab) to see how temperature is changed by elevation.  
[3D Zonal Crosstab](#3D-Zonal-Crosstab) to see how temperature is changed by elevation over time.

-----------


## Load data

The region of interest is a small area around the [Death Valley National Park
](https://www.google.com/maps/place/Death+Valley+National+Park/@36.5052209,-119.093306,538106m/data=!3m1!1e3!4m5!3m4!1s0x80c74b7776ae8a47:0xccc9f07c7bf2b054!8m2!3d36.5053891!4d-117.0794078!5m1!1e4), Skidoo, California, USA with an extent of `[-118, 36, -117, 37]` (EPSG:4326).

All data in this notebook has been loaded from stac items and has been coregistered so that they are well aligned. 

In [ ]:
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

from datashader.transfer_functions import shade, stack, Images

from xrspatial import hillshade
from xrspatial.classify import quantile
from xrspatial import zonal_crosstab

#### Load elevation data

In [ ]:
elevation = xr.open_rasterio('elevation.tif').sel(band=1)
elevation

Visualize the elevation raster.

In [ ]:
# Render the hillshade with a coloramp of the values applied on top
elevation_shaded = hillshade(elevation, azimuth=100, angle_altitude=50)
stack(
    shade(elevation_shaded, cmap=["white", "gray"]),
    shade(elevation, cmap=plt.get_cmap("terrain"), alpha=128)
)

#### Load temperature data

Temperature data in this example is collected for day time of 4 months of 2021 as specified in the dictionary below. 

In [ ]:
datetimes = {
    'March': '2021-03-31T00:00:00.000000000',
    'June': '2021-06-30T00:00:00.000000000',
    'September': '2021-09-30T00:00:00.000000000',
    'December': '2021-12-31T00:00:00.000000000'
}

MODIS Day Land Surface Temperature data is saved in Kelvin scale. Let's convert them to Fahrenheit that we're more familiar with.

In [ ]:
def kelvin_to_fahrenheit(k):
    return (k - 273.15) * 9/5 + 32 

Load day time temperature data.

In [ ]:
day_temp_data = xr.open_rasterio('day_temp_data.tif')

# replace 0s with NaNs
day_temp_data = day_temp_data.where(day_temp_data > 0, np.nan)

# convert to Fahrenheit scale
day_temp_data.data = kelvin_to_fahrenheit(day_temp_data.data)

day_temp_data

In [ ]:
day_temp_plots = day_temp_data.plot.imshow(cmap="magma", vmin=-10, vmax=125, col="band", size=4)
for ax, datetime in zip(day_temp_plots.axes.flat, datetimes.keys()):
    ax.set_title(datetime)

## 2D Zonal Crosstab

2D zonal crosstab works on two different 2D datasets, one for `zones`, and the other for `values`.

To define `zones` data, we'll use `xrspatial.classify.quantile` function to group elevation data into different classes. Each class will be a separate zone.

In [ ]:
n_elevation_classes = 10
zones = quantile(elevation, k=n_elevation_classes, name='Elevation Zones')

shaded_zones = shade(zones, cmap=plt.get_cmap("terrain"), how="linear")

In order to define the 2D `values` data, let's use day time temperature of March 2021 and categorize them into different classes. Each class contains temperature with similar values. We'll use `xrspatial.classify.quantile` once more to do this.

In [ ]:
day_temp_march = day_temp_data[0]

n_temp_classes = 10
classified_day_temp_march = quantile(day_temp_march, k=n_temp_classes, name='Temperature Classes (March)')
shaded_temp_classes_march = shade(classified_day_temp_march, cmap=plt.get_cmap("coolwarm"), how="linear")

Visualize the `zones` and `values` we've defined above.

In [ ]:
imgs = Images(shaded_zones, shaded_temp_classes_march)
imgs.num_cols = 2
imgs

Let's write a small util function to get range of each class/category in a categorical data array. From that, we can know exactly what range a elevation `zone` covers, and what range a temperature class is.

In [ ]:
# util function to calculate range of each class/bin

def bin_ranges(classified_data, original_data, unit):
    bins = np.unique(classified_data.data[~np.isnan(classified_data.data)])
    ranges = []
    for b in bins:
        bin_data = original_data.data[classified_data.data==b]
        ranges.append(f'{np.nanmin(bin_data):.1f}{unit} - {np.nanmax(bin_data):.1f}{unit}')
    return ranges

In [ ]:
elevation_ranges = bin_ranges(zones, elevation, 'm')
elevation_ranges

In [ ]:
temperature_ranges = bin_ranges(classified_day_temp_march, day_temp_march, 'F')
temperature_ranges

Now we got all the data we need. Let's run zonal crosstab to see how temperature is changed by elevation for March 2021.

In [ ]:
temp_march_by_elevation = zonal_crosstab(zones, classified_day_temp_march, agg='percentage')
temp_march_by_elevation['zone'] = elevation_ranges
temp_march_by_elevation.columns = ['Elevation', *temperature_ranges]
temp_march_by_elevation.set_index('Elevation', inplace=True)
temp_march_by_elevation

In the result table, each cell represents the percentage of a temperature class that falls within an elevation zone. It can easily be seen that the temperature decreases when the elevation increases and vice versa, the temperature increases when the elevation decreases.

In [ ]:
temp_march_by_elevation.plot(kind="bar", stacked=True);

## 3D Zonal Crosstab

3D zonal crosstab works on a 2D `zones` data array with a 3D `values` data array. There are many aggregation methods for calculating the cross tabulated stats between the 2 datasets: `[min, max, mean, sum, std, var, count]`.

In this example, let's see how the average temperature are changed by elevation over time for day time temperature.

In [ ]:
mean_day_temp = zonal_crosstab(zones, day_temp_data, agg='mean')
mean_day_temp['zone'] = elevation_ranges
mean_day_temp.columns = ['Elevation', *datetimes.keys()]
mean_day_temp.set_index('Elevation', inplace=True)

mean_day_temp

In [ ]:
mean_day_temp.reset_index().plot(
    x="Elevation", y=datetimes.keys(), kind="line", figsize=(10, 10)
);

Looking at the result of mean temperature for day time over the year, we can see that temperature reaches highest in June, and lowest in December. And it tends to decrease with increase in elevation height.